first we need to train a parametric-umap network for each dataset... (5 datasets x 2 dimensions)
For umap-learn, UMAP AE, Param. UMAP, PCA
- load dataset
- load network
- compute reconstruction MSE
- count time

In [1]:
# reload packages
%load_ext autoreload
%autoreload 2

### Choose GPU (this may not be needed on your computer)

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES='0'


In [3]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if len(gpu_devices)>0:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [4]:
import numpy as np
import pickle
import pandas as pd
import time
from umap import UMAP

In [5]:
from tfumap.umap import tfUMAP
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [6]:
from tqdm.autonotebook import tqdm

In [7]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

In [8]:
output_dir = MODEL_DIR/'projections' 

In [9]:
reconstruction_acc_df = pd.DataFrame(columns = ['method_', 'dimensions', 'dataset', 'MSE', 'MAE', 'MedAE', 'R2'])

In [10]:
reconstruction_speed_df = pd.DataFrame(columns = ['method_', 'dimensions', 'dataset', 'embed_time', 'recon_time', 'speed', 'nex'])

### MNIST

In [11]:
dataset = 'macosko2015'
dims = [50]

##### load dataset

In [12]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

#dataset_address = 'http://file.biolab.si/opentsne/macosko_2015.pkl.gz'
# https://opentsne.readthedocs.io/en/latest/examples/01_simple_usage/01_simple_usage.html
# also see https://github.com/berenslab/rna-seq-tsne/blob/master/umi-datasets.ipynb

import gzip
import pickle

with gzip.open(DATA_DIR / 'macosko_2015.pkl.gz', "rb") as f:
    data = pickle.load(f)

x = data["pca_50"]
y = data["CellType1"].astype(str)

print("Data set contains %d samples with %d features" % x.shape)

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=.1, random_state=42)

np.shape(X_train)

n_valid = 10000
X_valid = X_train[-n_valid:]
Y_valid = Y_train[-n_valid:]
X_train = X_train[:-n_valid]
Y_train = Y_train[:-n_valid]

X_train_flat = X_train

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

Y_train = enc.fit_transform([[i] for i in Y_train]).flatten()

X_train_flat = X_train
X_test_flat = X_test

50000 10000 10000


### AE 

##### 2 dims

In [13]:
load_loc = output_dir / dataset / 'autoencoder' 

In [14]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [15]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [16]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [17]:
X_recon = decoder(encoder(X_test)).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [18]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [19]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['AE', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,mnist,0.035955,0.080954,0.040411,0.048519


##### 64 dims

In [20]:
load_loc = output_dir / dataset / '64' / 'autoencoder' 

In [21]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [22]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [23]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [24]:
X_recon = decoder(encoder(X_test)).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [25]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [26]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['AE', 64, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,mnist,0.035955,0.080954,0.040411,0.048519
1,AE,64,mnist,0.002684,0.014447,0.001088,-11.882811


### Network

##### 2 dims

In [27]:
load_loc = output_dir / dataset / 'recon-network' 

In [28]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "network",
    batch_size = 100,
    dims = dims
)

In [29]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [30]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [31]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = encoder(X_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = decoder(z_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    recon_time = end_time - start_time
    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "network",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.22339396108873188
seconds:  1.8415809869766235
seconds:  0.16762002813629806
seconds:  2.1464138380251825
seconds:  0.18435529596172273
seconds:  1.7676494740881026
seconds:  0.16501359199173748
seconds:  1.8168301659170538
seconds:  0.17989090480841696
seconds:  1.7512574479915202
seconds:  0.18680215603671968
seconds:  1.7960831520613283
seconds:  0.17951037385500968
seconds:  1.8558823419734836
seconds:  0.1869460209272802
seconds:  1.7520582810975611
seconds:  0.17780991503968835
seconds:  1.7483326031360775
seconds:  0.1802855080459267
seconds:  1.8194242720492184



In [32]:
with tf.device('/CPU:0'):
    n_repeats = 10
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        z_test = encoder(X_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        embed_time = end_time - start_time

        start_time = time.monotonic()
        x_test_recon = decoder(z_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        recon_time = end_time - start_time
        reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
            "network-cpu",
            2,
            dataset,
            embed_time,
            recon_time,
            embed_time + recon_time,
        len(X_test_flat)
        ]

seconds:  0.1954057370312512
seconds:  1.7845982878934592
seconds:  0.17725891899317503
seconds:  1.7594706469681114
seconds:  0.16890336596406996
seconds:  1.744894202100113
seconds:  0.18498307396657765
seconds:  1.7331227699760348
seconds:  0.17813769401982427
seconds:  1.7646897889208049
seconds:  0.17754531698301435
seconds:  1.7954524240922183
seconds:  0.18359711300581694
seconds:  1.7532826971728355
seconds:  0.1837238969746977
seconds:  1.7299502179957926
seconds:  0.18665999313816428
seconds:  1.7621698249131441
seconds:  0.180573825025931
seconds:  1.7621062039397657



In [33]:
X_recon = decoder(encoder(X_test)).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [34]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [35]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['network', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,mnist,0.035955,0.080954,0.040411,0.048519
1,AE,64,mnist,0.002684,0.014447,0.001088,-11.882811
2,network,2,mnist,0.037375,0.083731,0.043021,0.219778


##### 64 dims

In [36]:
load_loc = output_dir / dataset / '64' / 'recon-network' 

In [37]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [38]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [39]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [40]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = encoder(X_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = decoder(z_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    recon_time = end_time - start_time
    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "network",
        64,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.22384602506645024
seconds:  1.8446767658460885
seconds:  0.1787384720519185
seconds:  1.7751228117849678
seconds:  0.1825248021632433
seconds:  1.74839371512644
seconds:  0.17339679598808289
seconds:  1.7577623568940908
seconds:  0.17435285518877208
seconds:  1.719195165205747
seconds:  0.18624946009367704
seconds:  1.750795803964138
seconds:  0.1807236298918724
seconds:  1.7611078438349068
seconds:  0.18094121594913304
seconds:  1.7966214779298753
seconds:  0.18616919801570475
seconds:  1.6974553519394249
seconds:  0.17311725788749754
seconds:  1.738615829963237



In [41]:
with tf.device("/CPU:0"):
    n_repeats = 10
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        z_test = encoder(X_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        embed_time = end_time - start_time

        start_time = time.monotonic()
        x_test_recon = decoder(z_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        recon_time = end_time - start_time
        reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
            "network-cpu",
            64,
            dataset,
            embed_time,
            recon_time,
            embed_time + recon_time,
        len(X_test_flat)
        ]

seconds:  0.2117242820095271
seconds:  1.7556050338316709
seconds:  0.1716506250668317
seconds:  1.7858995459973812
seconds:  0.19122329517267644
seconds:  1.7819799319840968
seconds:  0.18375799781642854
seconds:  1.7912935130298138
seconds:  0.19510670891031623
seconds:  1.789770547999069
seconds:  0.1889065180439502
seconds:  1.743673485936597
seconds:  0.17873317119665444
seconds:  1.7829772110562772
seconds:  0.17203829693607986
seconds:  1.7606663720216602
seconds:  0.18351537105627358
seconds:  1.7199972269590944
seconds:  0.17744691390544176
seconds:  1.691242909990251



In [42]:
reconstruction_speed_df

,method_,dimensions,dataset,embed_time,recon_time,speed,nex
0,network,2,mnist,0.223394,1.841581,2.064975,10000
1,network,2,mnist,0.167620,2.146414,2.314034,10000
2,network,2,mnist,0.184355,1.767649,1.952005,10000
3,network,2,mnist,0.165014,1.816830,1.981844,10000
4,network,2,mnist,0.179891,1.751257,1.931148,10000
5,network,2,mnist,0.186802,1.796083,1.982885,10000
6,network,2,mnist,0.179510,1.855882,2.035393,10000
7,network,2,mnist,0.186946,1.752058,1.939004,10000
8,network,2,mnist,0.177810,1.748333,1.926143,10000
9,network,2,mnist,0.180286,1.819424,1.999710,10000


In [43]:
X_recon = decoder(encoder(X_test)).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [44]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [45]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['network', 64, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,mnist,0.035955,0.080954,0.040411,0.048519
1,AE,64,mnist,0.002684,0.014447,0.001088,-11.882811
2,network,2,mnist,0.037375,0.083731,0.043021,0.219778
3,network,64,mnist,0.031332,0.073505,0.033229,-0.376026


#### UMAP-learn

##### 2 dims

In [46]:
embedder = UMAP(n_components = 2, verbose=True)
z_umap = embedder.fit_transform(X_train_flat)

UMAP(dens_frac=0.0, dens_lambda=0.0, verbose=True)
Construct fuzzy simplicial set
Fri Jul 17 12:09:56 2020 Finding Nearest Neighbors
Fri Jul 17 12:09:56 2020 Building RP forest with 16 trees
Fri Jul 17 12:09:58 2020 parallel NN descent for 16 iterations
	 0  /  16
	 1  /  16
	 2  /  16
	 3  /  16
	 4  /  16
Fri Jul 17 12:10:08 2020 Finished Nearest Neighbor Search
Fri Jul 17 12:10:10 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Fri Jul 17 12:10:55 2020 Finished embedding


In [47]:
x_test_samples= []
x_test_recon_samples= []
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    embed_time = end_time - start_time

    nex = 10 # it would take far too long to reconstruct the entire dataset
    samp_idx = np.random.randint(len(z_test),  size= nex)
    sample = np.array(z_test)[samp_idx]
    x_test_samples.append(samp_idx)
    start_time = time.monotonic()
    x_test_recon = embedder.inverse_transform(sample);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    recon_time = (end_time - start_time)*len(z_test)/nex

    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "umap-learn",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]
    x_test_recon_samples.append(x_test_recon)

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  18.49718587216921
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  56.08238297794014
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  6.02832709508948
	completed  0  /  10

In [48]:
x_recon = np.concatenate(x_test_recon_samples)

In [49]:
x_real = np.array(X_test_flat)[np.concatenate(x_test_samples)]

In [50]:

MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['umap-learn', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,mnist,0.035955,0.080954,0.040411,0.048519
1,AE,64,mnist,0.002684,0.014447,0.001088,-11.882811
2,network,2,mnist,0.037375,0.083731,0.043021,0.219778
3,network,64,mnist,0.031332,0.073505,0.033229,-0.376026
4,umap-learn,2,mnist,0.035111,0.092489,0.063589,-0.409136


##### PCA

##### 2 dims


In [51]:
pca = PCA(n_components=2)
z = pca.fit_transform(X_train_flat)

In [52]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = pca.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = pca.inverse_transform(z_test);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    recon_time = (end_time - start_time)

    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "pca",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.010590387973934412
seconds:  0.0206915820017457
seconds:  0.010562208015471697
seconds:  0.019900169223546982
seconds:  0.011132943909615278
seconds:  0.013940805103629827
seconds:  0.01633631600998342
seconds:  0.013821001863107085
seconds:  0.011437592096626759
seconds:  0.01402763812802732
seconds:  0.01109364302828908
seconds:  0.014068078948184848
seconds:  0.010818545008078218
seconds:  0.014045408926904202
seconds:  0.010884166928008199
seconds:  0.014337887056171894
seconds:  0.011094152927398682
seconds:  0.014425779925659299
seconds:  0.0168096290435642
seconds:  0.014675597194582224



In [53]:
X_recon = pca.inverse_transform(pca.transform(X_test_flat))
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['pca', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,mnist,0.035955,0.080954,0.040411,0.048519
1,AE,64,mnist,0.002684,0.014447,0.001088,-11.882811
2,network,2,mnist,0.037375,0.083731,0.043021,0.219778
3,network,64,mnist,0.031332,0.073505,0.033229,-0.376026
4,umap-learn,2,mnist,0.035111,0.092489,0.063589,-0.409136
5,pca,2,mnist,0.055667,0.130651,0.112987,0.145549


### Save

In [57]:
save_loc = DATA_DIR / 'reconstruction_speed' / (dataset + '.pickle')
ensure_dir(save_loc)
reconstruction_speed_df.to_pickle(save_loc)

In [58]:
save_loc = DATA_DIR / 'reconstruction_acc' / (dataset + '.pickle')
ensure_dir(save_loc)
reconstruction_acc_df.to_pickle(save_loc)